## Lab | Single page scraping

#### 1. import libraries

In [110]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [111]:
#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p > cite

#### 2. url: we start with the 'second' page. Show that you can start whenever you want


In [112]:
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

#### 3. download html with a get request

In [113]:
response = requests.get(url)
response.status_code # 200 status code means OK!

200

#### 4.1. parse html (create the 'soup')

In [114]:
soup = BeautifulSoup(response.content, "html.parser")

In [115]:
soup.title

<title>iTunes Top 100 Songs Chart 2022</title>

##### Buscamos el título

In [116]:
#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p > cite

In [117]:
soup.select("#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p > cite")

[<cite class="title">Unholy</cite>]

In [118]:
soup.select("cite.title")[:3]

[<cite class="title">Unholy</cite>,
 <cite class="title">Eagle (feat. KB)</cite>,
 <cite class="title">Everywhere</cite>]

##### Buscamos el artista

In [119]:
#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p > em

In [120]:
soup.select("#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p > em")

[<em class="artist">Sam Smith &amp; Kim Petras</em>]

In [121]:
soup.select("em.artist")[:3]

[<em class="artist">Sam Smith &amp; Kim Petras</em>,
 <em class="artist">Transformation Worship</em>,
 <em class="artist">Fleetwood Mac</em>]

##### Buscamos el ranking

In [122]:
#chart-position-1 > div.cover-art.col-xs-12.col-sm-4 > p

In [123]:
soup.select("#chart-position-1 > div.cover-art.col-xs-12.col-sm-4 > p")

[<p class="chart-position">1</p>]

In [124]:
soup.select("p.chart-position")[:3]

[<p class="chart-position">1</p>,
 <p class="chart-position">2</p>,
 <p class="chart-position">3</p>]

##### Buscamos el género

In [125]:
#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > ul > li:nth-child(1) > a

In [126]:
soup.select("#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > ul > li:nth-child(1) > a")

[<a href="/music/charts/top-pop-songs.php">Pop</a>]

In [127]:
soup.select("a.Pop")

[]

## making beautiful soups into beautiful tables

In [128]:
#initialize empty lists
rank = []
title = []
artist = []

# define the number of iterations of our for loop
# by checking how many elements are in the retrieved result set
# (this is equivalent but more robust than just explicitly defining 250 iterations)
num_iter = len(soup.select("cite.title"))

rango = soup.select("p.chart-position")
canciones = soup.select("cite.title")
artistas = soup.select("em.artist")
# género = pass
# spanlist = soup.select("td.titleColumn span.secondaryInfo")
# iterate through the result set and retrive all the data
for i in range(num_iter):
    rank.append(rango[i].get_text())
    title.append(canciones[i].get_text())
    artist.append(artistas[i].get_text())

# print(title)
# print(artist)

#### Scraping/treating Genre and Release date
The website stores all details on parent components but if it is a new release the first element is 
"first release", otherwise is the genre.
So genre, release date are on 0 and 1 index regularly, 1 and 2 if it's a new release.

In [138]:
# list to store all elements
multi = []

for i in range(100):
    multi.append(soup.select("#chart-position-"+str(i+1)+" > div.chart-content.col-xs-12.col-sm-8 > ul > li"))

# genre and release date sublists 
genre = []
release = []

# release date are on 0 and 1 index regularly, 1 and 2 if it's a new release
for element in multi:
    if element[0].get_text() == 'New Release':
        genre.append(element[1].get_text().split('Genre: ')[1])
        release.append(element[2].get_text().split('Release Date: ')[1])
    else:
        genre.append(element[0].get_text().split('Genre: ')[1])
        release.append(element[1].get_text().split('Release Date: ')[1])

In [139]:
artist_song = pd.DataFrame({"rank":rank,
                        "title":title,
                        "artist":artist,
                        "genre":genre,
                        "release_date":release
                        })
artist_song.to_csv("top100songs",index=False)

In [140]:
artist_song.head()

,rank,title,artist,genre,release_date
0,1,Unholy,Sam Smith & Kim Petras,Pop,"September 22, 2022"
1,2,Eagle (feat. KB),Transformation Worship,Hip-Hop / Rap,"October 7, 2022"
2,3,Everywhere,Fleetwood Mac,Rock,"April 14, 1987"
3,4,I'm Good (Blue),David Guetta & Bebe Rexha,Dance,"August 26, 2022"
4,5,wait in the truck,HARDY & Lainey Wilson,Country,"August 26, 2022"


Function for recommendation

In [131]:
from random import randint


def recomendacion_musical():
    input_song = input("Introduce the name of a song: ")
    for i in artist_song['title']:
        if input_song in i:
            return(artist_song['title'][randint(1,len(artist_song['title']))])

In [132]:
recomendacion_musical()

'500 Miles'

## Lab | Multiple page scraping

## Respectful scraping:

In [133]:
from time import sleep
from random import randint
import numpy as np

In [134]:
# for i in range(5):
#     print(i)
#     wait_time = randint(1,4000)
#     print("I will sleep for " + str(wait_time/1000) + " seconds.")
#     sleep(wait_time/1000)

## Assembling the script to send and store multiple requests